In [25]:
%pwd

'c:\\Users\\nipun\\Desktop\\Medchatbot'

In [20]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os 

In [21]:
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [26]:
extracted_data=load_pdf_file(data='Data/')

In [27]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [28]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5860


In [31]:
from langchain.embeddings import HuggingFaceEmbeddings

In [32]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings


In [33]:
embeddings = download_hugging_face_embeddings()

C:\Users\nipun\AppData\Local\Temp\ipykernel_25384\1196424635.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [34]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [35]:
from dotenv import load_dotenv
load_dotenv()

True

In [52]:
GROQ_API_KEY=os.environ.get('GROQ_API_KEY')

In [51]:
import os
from pinecone import Pinecone, ServerlessSpec

# Your Pinecone API key
PINECONE_API_KEY = "pcsk_6hfbZM_PyDhaPVXZs1H5ddoLnsEWx7iNai3SZR8Mfbgsf59yEKteEmXsiatUEu5q2RPML2"  # Replace with your actual API key

# Initialize Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY)

# Define the index name
index_name = "medbot"

# Create the index if it doesn't exist
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,  # Adjust based on your embeddings
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

print(f"Index '{index_name}' is ready!")


Index 'medbot' is ready!


In [ ]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [39]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [40]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [41]:
docsearch

In [42]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [45]:
retrieved_docs = retriever.invoke("What is Cancer?")

In [46]:
retrieved_docs

[Document(id='9e6b1db4-d67f-48e8-8d19-f9d5f3cd642d', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 184.0, 'page_label': '185', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='KEY TERMS\nBiopsy—A procedure in which a small piece of\nbody tissue is removed and examined under a\nmicroscope for cancer.\nChemotherapy —A cancer treatment in which\ndrugs delivered into the blood stream kill cancer\ncells or make them more vulnerable to radiation\ntherapy.\nHuman papillomavirus (HPV)—A virus with many\nsubtypes, some of which cause cell changes that\nincrease the risk of certain cancers.\nHuman immunodeficiency virus (HIV)—The virus\nthat causes acquired immune deficiency syn-'),
 Document(id='a5733cb0-af14-4755-9c6a-56c7a46c034e', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 5

In [55]:
import os
from langchain_groq import ChatGroq

# Set API key directly
api_key = "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"

llm = ChatGroq(model_name="mixtral-8x7b-32768", temperature=0.4, max_tokens=500, api_key=api_key)


In [56]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


In [57]:
# Create the question-answering chain
question_answer_chain = create_stuff_documents_chain(llm, prompt)

# Assume `retriever` is already defined elsewhere
rag_chain = create_retrieval_chain(retriever, question_answer_chain)  

In [ ]:
# Invoke the RAG pipeline
response = rag_chain.invoke({"input": "what are the clinical features of fanconi’s syndrome?"})
print(response["answer"])

The context does not provide information on the specific genetic mutation that causes Fanconi's anemia. It only mentions that hereditary aplastic anemia occurs in Fanconi's anemia, among other syndromes.
